In [ ]:
push!(LOAD_PATH, "/home/amir/Work/gdmrg/src/")
import Tmp
using Plots
using LaTeXStrings

In [ ]:
sz_half = Tmp.sz_half;
sp_half = Tmp.sp_half;
sm_half = Tmp.sm_half;

In [ ]:
### loading example files for illustration ###
#include("example-01-compareHeisVsRandom.jl")

In [ ]:
reload("Tmp")
include("test/runtests.jl")

In [ ]:
flatten(list) = collect(Base.Iterators.flatten(list))

Lx = 12

Hhs = Tmp.xxz(Lx)
eheis, vheis = eigs(Hhs, nev=1, which=:SR)
mpsheis = Tmp.MPS(Lx, 2, vheis[:,1])
measheis_z = flatten(Tmp.measure(mpsheis, sz_half, collect(1:Lx)))
measheis_zz = Tmp.measure(mpsheis, [sz_half, sz_half])
measheis_pm = Tmp.measure(mpsheis, [sp_half, sm_half])

corr_matrix = Tmp.correlation_matrix(Tmp.hamiltonian_matrix(Lx, 1.0, 1.), div(Lx,2))
gmps = Tmp.makeGMPS!(corr_matrix, 1.e-8)

twoflavormps = Tmp.generateMPS_twoflavors(gmps, 400)
@show twoflavormps.dims
mpsgutz = Tmp.gutzwiller_project(twoflavormps)

@show gutznorm = real(Tmp.norm(mpsgutz)[1,1])

measgutz_z = flatten(Tmp.measure(mpsgutz, sz_half, collect(1:Lx)))
measgutz_zz = Tmp.measure(mpsgutz, [sz_half, sz_half])/gutznorm
measgutz_pm = Tmp.measure(mpsgutz, [sp_half, sm_half])/gutznorm
mpo = Tmp.MPO{Float64}(Lx, 2)

@show eheis, Tmp.measure(mpsgutz, mpo)/gutznorm, Tmp.measure(mpsheis, mpo)
@show Tmp.overlap(mpsheis, mpsgutz)/sqrt(gutznorm)

p1 = plot(measheis_zz[1:Lx-1], label="Heisenberg", title=L"\langle s^z,s^z \rangle")
plot!(p1, real.(measgutz_zz[1:Lx-1]), label="Gutzwiller")
#plot!(p1, imag.(measgutz_zz[1:Lx-1]), label="Gutzwiller imaginary")

p2 = plot(real.(measheis_pm[1:Lx-1]), label="Heisenberg", title=L"\langle s^+,s^- \rangle")
plot!(p2, real.(measgutz_pm[1:Lx-1]), label="Gutzwiller")
#plot!(p2, imag.(measgutz_pm[1:Lx-1]), label="Gutzwiller imaginary")

@show mpsgutz.dims

p3 = plot(real.(measheis_z), label="Heisenberg", title="bond texture")
plot!(p3, real.(measgutz_z), label="Gutzwiller")

vneheis = Tmp.entropy(Tmp.entanglements!(mpsheis))
vnegutz = Tmp.entropy(Tmp.entanglements!(mpsgutz))

p4 = plot(vneheis, label="Heisenberg", title="Entropies")
plot!(p4, label="Gutzwiller", vnegutz)

plot(p1,p2,p3,p4, layout=(2,2))